In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/src')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00


In [ ]:
import evaluate
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
accuracy = evaluate.load("accuracy")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def preprocess_function(examples):
  input = examples["acronym_token"] + tokenizer.sep_token + examples["expansion"] + tokenizer.sep_token + examples["sentence"]
  return tokenizer(input, truncation=True)

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
# load data
training_set = pd.read_parquet('../dataset/parquet/train.parquet')
testing_set = pd.read_parquet('../dataset/parquet/test.parquet')
training_set['label'] = training_set['label'].astype(int)
testing_set['label'] = testing_set['label'].astype(int)

In [ ]:
# build DatasetDict
train_dataset = Dataset.from_pandas(training_set)
test_dataset = Dataset.from_pandas(testing_set)
ad_dataset = DatasetDict({
  'train': train_dataset,
  'test': test_dataset
})

ad_dataset

DatasetDict({
    train: Dataset({
        features: ['acronym_token', 'expansion', 'sentence', 'label'],
        num_rows: 22720
    })
    test: Dataset({
        features: ['acronym_token', 'expansion', 'sentence', 'label'],
        num_rows: 5566
    })
})

In [ ]:
# tokenize
tokenized_ad_dataset = ad_dataset.map(preprocess_function, batched=False)
tokenized_ad_dataset

Map:   0%|          | 0/22720 [00:00<?, ? examples/s]

Map:   0%|          | 0/5566 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['acronym_token', 'expansion', 'sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 22720
    })
    test: Dataset({
        features: ['acronym_token', 'expansion', 'sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5566
    })
})

In [ ]:
training_args = TrainingArguments(
  output_dir="model",
  learning_rate=1e-5,
  per_device_train_batch_size=32,
  per_device_eval_batch_size=32,
  num_train_epochs=7,
  weight_decay=0.001,
  warmup_ratio=0.05,
  logging_steps=710,
  save_total_limit=7,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  load_best_model_at_end=True,
)

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_ad_dataset["train"],
  eval_dataset=tokenized_ad_dataset["test"],
  tokenizer=tokenizer,
  data_collator=data_collator,
  compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.447400,0.327585,0.866152
2,0.295500,0.288485,0.889148
3,0.243000,0.289060,0.887531
4,0.207700,0.277957,0.890586
5,0.179000,0.289795,0.899389
6,0.162100,0.300069,0.897952
7,0.146500,0.308901,0.898670


TrainOutput(global_step=4970, training_loss=0.24018360805703362, metrics={'train_runtime': 2431.6875, 'train_samples_per_second': 65.403, 'train_steps_per_second': 2.044, 'total_flos': 6494197660685568.0, 'train_loss': 0.24018360805703362, 'epoch': 7.0})